In [8]:
import numpy as np
import pandas as pd

from pathlib import Path
from colorama import Fore, Style
from dateutil.parser import parse

from power.params import *
from power.ml_ops.data import get_data_with_cache, load_data_to_bq
# from power.ml_ops.model import initialize_model, compile_model, train_model, evaluate_model
from power.ml_ops.registry import load_model, save_model, save_results
# from power.ml_ops.registry import mlflow_run, mlflow_transition_model

2024-03-08 16:06:03.673079: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-08 16:06:03.832001: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-08 16:06:03.834895: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-08 16:06:06.719132: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
input_prediction_date = "2019-12-10 00:00:00"
# make api call
base_url = "http://127.0.0.1:8000"
endpoint_model = "/predict/previous_value"
endpoint_data = "/extract_data"
url_model = f"{base_url}{endpoint_model}"
url_data = f"{base_url}{endpoint_data}"

params ={
    'input_date':input_prediction_date
    }

response_model = requests.get(url_model, params=params).json()
model_df = pd.DataFrame(response_model)

response_data = requests.get(url_data, params=params).json()
data_df = pd.DataFrame(response_data)

In [6]:
response_model

{'2019-12-10 00:00:00': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.01,
  0.068,
  0.173,
  0.402,
  0.419,
  0.349,
  0.165,
  0.012,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0]}

In [9]:
print(Fore.MAGENTA + "\n⭐️ Use case: train" + Style.RESET_ALL)
print(Fore.BLUE + "\nLoading preprocessed validation data..." + Style.RESET_ALL)


# Load processed data using `get_data_with_cache` in chronological order

query = f"""
    SELECT *
    FROM {GCP_PROJECT}.{BQ_DATASET}.processed_pv
    ORDER BY utc_time
"""

data_processed_cache_path = Path(LOCAL_DATA_PATH).joinpath("processed", f"processed_pv.csv")
data_processed = get_data_with_cache(
    gcp_project=GCP_PROJECT,
    query=query,
    cache_path=data_processed_cache_path,
    data_has_header=True
)


⭐️ Use case: train

Loading preprocessed validation data...

Load data from BigQuery server...
✅ Data loaded, with shape (376944, 3)
